In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

# Define image dimensions
img_width, img_height = 224, 224

# Define AlexNet architecture
alexnet_model = Sequential([
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(256, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(384, (3, 3), activation='relu'),
    Conv2D(384, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(1000, activation='softmax')  # 1000 classes in ImageNet, adjust for your task
])

# Load pre-trained weights (if available)
# alexnet_model.load_weights('path_to_pretrained_weights.h5')

# Load VGG16 pre-trained model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Define data generators
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Set your training and testing directories
train_dir = "C:\\Users\\tumma\\Desktop\\skincancerdetection\\train"
test_dir = "C:\\Users\\tumma\\Desktop\\skincancerdetection\\test"

# Define batch size
batch_size = 32

# Generate training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,  # This means our generator will only yield batches of data, no labels
    shuffle=False)  # Important to keep the same order as our labels

# Generate testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

# Extract features using AlexNet
alexnet_X_train_features = alexnet_model.predict(train_generator, verbose=1)
alexnet_X_test_features = alexnet_model.predict(test_generator, verbose=1)

# Extract features using VGG16
vgg_X_train_features = vgg_model.predict(train_generator, verbose=1)
vgg_X_test_features = vgg_model.predict(test_generator, verbose=1)





Found 11879 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
63/63 [==============================] - 527s 8s/step


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Concatenate
from keras.callbacks import EarlyStopping

# Concatenate the features from AlexNet and VGG16
concatenated_X_train_features = Concatenate()([Flatten()(alexnet_X_train_features), Flatten()(vgg_X_train_features)])
concatenated_X_test_features = Concatenate()([Flatten()(alexnet_X_test_features), Flatten()(vgg_X_test_features)])

# Get the actual class labels for training and testing data
y_train = train_generator.classes
y_test = test_generator.classes

# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(concatenated_X_train_features.shape[1],)),  
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary classification, adjust as needed
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Assuming binary classification
              metrics=['accuracy'])

# Print model summary
model.summary()

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(concatenated_X_train_features, y_train, epochs=10, batch_size=32,
                    validation_data=(concatenated_X_test_features, y_test), callbacks=[early_stop])

# Evaluate the model
test_loss, test_acc = model.evaluate(concatenated_X_test_features, y_test)
print('Test accuracy:', test_acc)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               3339392   
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 1)               

In [4]:
from keras.preprocessing import image
import numpy as np

# Define the function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match batch size
    img_array /= 255.  # Normalize pixel values
    return img_array

# Define the function to predict the class of the image
def predict_image_class(img_path):
    preprocessed_img = preprocess_image(img_path)
    # Extract features using both AlexNet and VGG16
    alexnet_features = alexnet_model.predict(preprocessed_img)
    vgg_features = vgg_model.predict(preprocessed_img)
    # Flatten the feature maps
    alexnet_flattened_features = np.ravel(alexnet_features)
    vgg_flattened_features = np.ravel(vgg_features)
    # Concatenate the flattened features
    concatenated_features = np.concatenate([alexnet_flattened_features, vgg_flattened_features])
    # Make prediction using the concatenated features
    prediction = model.predict(np.expand_dims(concatenated_features, axis=0))
    # Interpret predictions
    if prediction > 0.5:
        return "Malignant"
    else:
        return "Benign"

# Test the function on an example image
test_image_path = "C:\\Users\\tumma\\Desktop\\skincancerdetection\\test\\Benign\\6315.jpg"  # Provide the path to your test image
predicted_class = predict_image_class(test_image_path)
print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 127ms/step
Predicted class: Benign
